cambodia jsc

In [102]:
import pandas as pd
from fuzzywuzzy import process,fuzz
from tqdm import tqdm,trange
import re
import sqldf

def strProcess(s:str, aggresive=False):
    s=s.strip()
    s=s.replace("Co.,","Co.") # Co., -> Co.
    s=s.lower() #小写 
    s=re.sub(r'company limit[^\s]{0,}','co., ltd',s)
    s=re.sub(r'limited[^\w]{0,}$', 'ltd', s)
    s=s.replace("&"," and ").replace("–","-").replace("mfg.","manufacturing").replace("ltd co","")
    s=re.sub(r"joint\s{1,}stock\s{1,}co[^\s]{1,}",'jsc', s) #joint stock co -> jsc
    if "former" not in s: 
        s=re.sub(r'\(.*?\)', '', s) # 括号内文字删除，除了former的情况
    s=re.sub(r'\s{2,}', ' ', s) #多个空格 -> 单个

    if aggresive==True:
        s=re.sub(r'-\s.{2,}',"",s) # - xxx -> 删除
        s=re.sub(r'co\.?\,?\s?ltd\.?',"",s) # co ltd 删除
    s=re.sub(r'[^\w\d)]*$',"",s) #处理结尾除右括号的特殊符号
    s=re.sub("[^\w\s.,()-]",'',s) #处理除.,()-空格外的特殊符号
    if s.startswith("pt"): 
        s=re.sub(r'^pt\.?\s', '', s)+" pt" # 处理pt: 开头转结尾
    return s

def getBWDuplicates(verbose=False, col="FactoryName"):
    global bw
    nameCounts=bw.groupby("FactoryBWID")[col].nunique()
    susNames=nameCounts[nameCounts > 1].index
    opdfv=bw[bw['FactoryBWID'].isin(susNames)]
    opdf=opdfv.groupby("FactoryBWID")[col].unique().apply(list).to_dict()
    if verbose==False:
        return opdf
    else:
        return opdfv
    
def getFormer(s):
    # works bcuz formerly only has 2 situation: ends with right bracket or with nothing, but seprated by semicolumn
    ## 'Exploitasi Energi Indonesia Tbk; PT (formerly Central Korporindo Internasional; CENKO'
    try:
        return re.findall(r"formerly.*[);]",s.lower())[0].replace("formerly","").strip()[:-1].strip()
    except:
        return s
    
def queryOrigName(df:str, s, rgx=False):
    if df=="bw":
        global bw
        return bw[bw["FactoryName"].str.lower().str.contains(s, regex=rgx)]
    elif df=="ci":
        global ci
        return ci[ci["company_name"].str.lower().str.contains(s,regex=rgx)]
    else:
        return None

def getFirst2(s):
    if len(s.split(" "))<2:
        return s
    else:
        return " ".join(s.split(" ")[:2])

In [14]:
bw=pd.read_csv("../w7/bw.csv")
ci=pd.read_csv("../w7/cif.csv").drop("former",axis=1)
bwFacts=list(bw.FactoryName.unique()) #len: 2094
ciFacts=list(ci.company_name.unique()) #len: 15339

In [96]:
bw["cName"]=bw["FactoryName"].apply(getFormer).apply(strProcess, aggresive=1)
ci["cName"]=ci["company_name"].apply(getFormer).apply(strProcess, aggresive=1)

---

In [106]:
queryOrigName('bw',"former[^\w]", True)["FactoryName"].unique()

array(['Win Shingtex (Cambodia) Co., Ltd. (former Shing Tex Co., Ltd)',
       'Zhong Yov Co., Ltd. (former Kun Mao Hsing Co., Ltd)',
       'Wear Mag Ltd. (Former)', 'AKH Fashions Limited (Former Factory)',
       'AKH Shirts Limited (Former Factory)',
       'Clifton Apparels Ltd. (Former)', 'Vision Garments Ltd (Former)',
       'KDS Apparels Ltd. (Former)',
       'Sonali Fabrics & Textile Mills (Pvt.) Ltd. (Sweater) (former factory)'],
      dtype=object)

In [110]:
queryOrigName('bw',"former")["FactoryName"].nunique()

77

In [122]:
first2=[]
for i in bwFacts1:
    if getFirst2(i) in ciFacts1:
        first2.append(i)

len(first2)

83

In [135]:
cnt=0
bwFacts2=[x for x in bwFacts1 if "former" not in x]
for i in tqdm(set(bwFacts2)-set(match1)):
    flag=False
    for j in ciFacts1:
        if getFirst2(i) in j:
            flag=True
            break
    if flag:
        continue
    else:
        cnt+=1

f"{cnt}/{len(bwFacts2)}"

  0%|          | 0/1543 [00:00<?, ?it/s]

100%|██████████| 1543/1543 [00:14<00:00, 105.82it/s]


'1291/1881'

---
# match 1:

In [116]:
bwFacts1=[strProcess(x,aggresive=1) for x in bw["cName"].unique()]
ciFacts1=[strProcess(x,aggresive=1) for x in ci["cName"].unique()]
len(set(bwFacts)),len(set(ciFacts))

(2091, 15339)

In [136]:
2091-1291-330

470

In [119]:
match1

['prima sejati sejahtera pt',
 'crystal martin ltd',
 'hung wah garment manufacturing ltd',
 'meroson',
 'hwashin print',
 'green vina',
 'winnie fashion',
 'kinh bac garment jsc',
 'eastex garment',
 'ftn vietnam',
 'vega textile ltd',
 'l and b indonesia pt',
 'masterindo jaya abadi pt',
 'manufacturing sportwear jsc',
 'hi-tech apparel',
 'fasic indonesia pt',
 'orchid garments ltd',
 'garbotex trading',
 'king lian garment factory',
 'mitra garindo perkasa pt',
 'sisophon hong seng sportswear',
 'buma apparel industry pt',
 'elite garments industries ltd',
 'alpha-fr international',
 'hoa sen phu tho garment',
 'cch indonesia pt',
 'citra abadi sejati pt',
 'gin-sovann fashion ltd',
 'global garment sourcing vietnam',
 'interstoff apparels ltd',
 'inko prima idaman apparel pt',
 'sansan saudaratex jaya pt',
 'crown exclusive wears ltd',
 'indesore sweater ltd',
 'new archid garment factory ltd',
 'wanshen clothing',
 'energypac fashions ltd',
 'divine intimates ltd',
 'tak son laun

In [59]:
match1=list(set(bwFacts1)&set(ciFacts1))
len(match1)

330

In [66]:
sqldf.run("select distinct(company_name), cName from ci where company_name in (select company_name from ci group by cName having count(distinct(company_name))>1)")

,company_name,cName
0,Fashion Forum Ltd,fashion forum ltd
1,Hutchison 3 Indonesia; PT [Please refer to Ind...,indosat tbk pt
2,Vinacomin - Coal Import Export Joint Stock Com...,vinacomin
3,Pelabuhan Indonesia IV; PT (Pelindo IV) [Pleas...,pelabuhan indonesia ii pt
4,International Manpower Supply and Trade JSC (S...,international manpower supply and trade jsc
5,PT Perkebunan Nusantara I (PTPN I),perkebunan nusantara i pt
6,PT Gorontalo Minerals,gorontalo minerals pt
7,KrisEnergy (Satria) Ltd,krisenergy ltd
8,AKM Knit Wear Ltd (AKM Knitwear Ltd),akm knit wear ltd
9,Saigon - Song lam Beer JSC,saigon


In [68]:
ci[ci["company_name"].str.lower().str.contains("please refer")]["company_name"].unique()

array(['Hutchison 3 Indonesia; PT [Please refer to Indosat Ooredoo Hutchison Tbk; PT (Indosat Ooredoo; formerly Indosat Tbk; PT)]',
       'Pelabuhan Indonesia IV; PT (Pelindo IV) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'Pelabuhan Indonesia III; PT (Pelindo III) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'Pelabuhan Indonesia I; PT (Pelindo I) [Please refer to Pelabuhan Indonesia; PT (Pelindo; formerly Pelabuhan Indonesia II; PT)]',
       'United Middle-East & Commodore Hotels Co; The [Please refer to Union Land Development Corp]'],
      dtype=object)

# match 2

In [69]:
match2={}
for i in tqdm(set(bwFacts1)-set(match1)):
    cand=[x for x in ciFacts1 if x[0]==i[0]]
    possibleMatch=[y for y in cand if fuzz.partial_ratio(y,i)>75]
    if len(possibleMatch)>0:
        match2[i]=possibleMatch
    # sorted(cand,key=lambda x : fuzz.partial_ratio(i,x),reversed=True)

100%|██████████| 1552/1552 [06:59<00:00,  3.70it/s]


In [70]:
len(match2)

893

In [71]:
{x:match2[x] for x in match2 if len(match2[x])==1}

{'standard textile jordan': ['sarah textile'],
 'universal design nicaragua, s.a': ['uslc de nicaragua'],
 'lecien corporation': ['lac viet computing corporation'],
 'yupoong vietnam': ['yazaki haiphong vietnam'],
 'shanta expression ltd': ['sonic ltd'],
 'vina korea': ['vina hk'],
 'fashion garments 2': ['fountain garments manufacturing ltd'],
 'm and j garment-printing-embroidery': ['m and j group'],
 'pacific sports haiti s.a': ['pacific sports haiti sa'],
 'sidney apparels llc': ['supershine apparels ltd'],
 'di seta': ['damai setiatama timber pt'],
 'prima il jo pt': ['ptg'],
 'cambo handsome ltd. 3': ['cambo handsome ltd'],
 'gunung abadi pt': ['gunung harta pt'],
 'lotus textile and garment': ['lianfa garment'],
 'mascot knits ltd': ['ms knit ltd'],
 'grupo alfatech,s.a': ['grupo gama'],
 'chun hing knitting international': ['cinlon international'],
 'horizon outdoor': ['horizon outdoor cambodia'],
 'jeacon garments ltd': ['jms garments ltd'],
 'nan kuang garment': ['ntn'],
 'qu

---
# match FNC, RRID

In [150]:
bw.drop(["CNRR","repriskID"],axis=1, inplace=True)

In [152]:
merged = pd.merge(bw, ci[['cName', 'company_name','repriskID']], on='cName', how='left')
bw['CNRR']=merged['company_name']
bw['repriskID']=merged['repriskID']
# sqldf.run("select ci.cName, ci.company_name, ci.repriskID from ci inner join bw on ci.cName=bw.cName")
bw

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID
0,12129,2,Hi-Tech Textile L.L.C.,Jordan,19/10/2015,2015,None,hi-tech textile l.l.c,NaN,NaN
1,17215,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2016,2016,None,hi-tech textile l.l.c,NaN,NaN
2,22596,2,Hi-Tech Textile L.L.C.,Jordan,27/09/2017,2017,None,hi-tech textile l.l.c,NaN,NaN
3,30133,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2018,2018,None,hi-tech textile l.l.c,NaN,NaN
4,36539,2,Hi-Tech Textile L.L.C.,Jordan,14/10/2019,2019,None,hi-tech textile l.l.c,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6495,8197,36,CODEVI,Haiti,2015-10-02 00:00:00,2015,None,codevi,CODEVI,118360.0
6496,8262,1540,Valdor Apparel Mfg S.A.,Haiti,27/01/2015,2015,None,valdor apparel mfg s.a,NaN,NaN
6497,8212,112,Casual Wear Apparel L.L.C.,Haiti,18/01/2015,2015,None,casual wear apparel l.l.c,NaN,NaN
6498,8214,139,One World Apparel S.A.,Haiti,14/01/2015,2015,None,one world apparel s.a,NaN,NaN


In [154]:
bw.repriskID.nunique()

328